In [222]:
with open('text.txt', 'r') as file:
    data = file.read().split('\n\n')

In [224]:
#сбор признаков
def get_features(data):
    features = {
        'isAbbreviation': [],
        'isNumber': [],
        'numberOfWords': [],
        'isNextLetterCapital': [],
        'isNextLetterNumber': []
    }
    
    for paragraph in data:
        if '.' not in paragraph:
            continue
        sentences = paragraph.split('.')

        for i, sentence in enumerate(sentences):

            if i == len(sentences) - 1: #граничное условие, последняя точка в абзаце всегда является разделяющей
                continue
            if not sentences[i+1]:
                continue
            sents.append(sentence)
            words = sentence.split(' ')
            lastWord = words[-1] #последнее слово в предложении
            nextSentence = sentences[i+1] #предложение, следующее за точкой

            if len(lastWord) <= 3:
                features['isAbbreviation'].append(1)
            else:
                features['isAbbreviation'].append(0)

            if lastWord.isnumeric():
                features['isNumber'].append(1)
            else:
                features['isNumber'].append(0)

            features['numberOfWords'].append(len(words))

            if nextSentence[0] == ' ':
                if nextSentence[1].upper() == nextSentence[1]:
                    features['isNextLetterCapital'].append(1)
                else:
                    features['isNextLetterCapital'].append(0)
            else:
                if nextSentence[0].upper() == nextSentence[0]:
                    features['isNextLetterCapital'].append(1)
                else:
                    features['isNextLetterCapital'].append(0)
            if nextSentence[0] == ' ':
                if nextSentence[1].isnumeric():
                    features['isNextLetterNumber'].append(1)
                else:
                    features['isNextLetterNumber'].append(0)
            else:
                if nextSentence[0].isnumeric():
                    features['isNextLetterNumber'].append(1)
                else:
                    features['isNextLetterNumber'].append(0)
    return features

In [225]:
features = get_features(data)

In [226]:
import pandas as pd
df_test = pd.DataFrame(features)

In [227]:
target_test = [0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,
         0, 0,0, 0,0, 0,0, 0,0,0,0,0,0,0,
         0,0,0,0,0,1,0,0,0,0,1, 1, 0,
         0,0,0,0,
         0,
         0,0,0,0,0,0,0,0,0,0,0,1,
         0,0,0,0,
         0,0,0,0,0, 0,1,0,0,0,0,0,0,0,0,0,0,0,0,
          0,0,0,0,0,
          0,
          0,0,0,0,0,0,0,0,
          0,0,
          0,0,0,0,0,
          0,1,0,
          1,
          0,
          0,0,
          0,
          0,
          0
         ]

In [228]:
df_test['target'] = target_test

In [229]:
df_test.head()

,isAbbreviation,isNumber,numberOfWords,isNextLetterCapital,isNextLetterNumber,target
0,1,0,5,1,0,0
1,1,0,10,1,0,0
2,1,0,1,1,0,0
3,1,0,4,1,0,0
4,1,0,1,1,0,0


In [230]:
with open('text2.txt', 'r') as file:
    data2 = file.read().replace('...', '').split('\n\n')

In [231]:
target_train = [0,
                 0,0,0,0,0,0,0,0,0,0,1,
                  0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,
                  0,0,1, 0,0,0,0,1,1,1,1,1,1,1,1,1,1,
                  1, 0,0,1,0,0,0,
                  0,0,1,0,0,0,0,
                  0,0,0,1, 0,0,0,
                  0,
                  1,
                    0,1,0,0,
                   0,0,
                   1,0,0,1,0,1,
                   0,
                   0,0, 0,0,0,0,1,0,0,
                   0,
                   0,
                   0,
                  ]

In [232]:
features2 = get_features(data2)

In [233]:
df_train = pd.DataFrame(features2)

In [234]:
df_train['target'] = target_train

In [235]:
df_train.head()

,isAbbreviation,isNumber,numberOfWords,isNextLetterCapital,isNextLetterNumber,target
0,1,0,5,1,0,0
1,1,0,10,1,0,0
2,1,0,1,1,0,0
3,1,0,4,1,0,0
4,1,0,1,1,0,0


In [236]:
parametres = { 'criterion': ['gini', 'entropy'],
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

In [242]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

X_train = df_train.drop('target', axis=1)
y_train = target_train
grid = GridSearchCV(DecisionTreeClassifier(), parametres, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2)})

In [243]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 1,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [244]:
X_test = df_test.drop('target', axis=1)
y_test = target_test


from sklearn.metrics import f1_score
f1_score(y_test, grid.predict(X_test))

0.8333333333333333

In [245]:
from sklearn.neighbors import KNeighborsClassifier
nn = KNeighborsClassifier(n_neighbors=5)
nn.fit(X_train, y_train)

KNeighborsClassifier()

In [246]:
f1_score(y_test, nn.predict(X_test))

0.29629629629629634

In [248]:
parametres_forest = { 'n_estimators': range (20,100, 10),
                      'max_depth': range (1,13, 2),
                      'min_samples_leaf': range (1,8),
                      'min_samples_split': range (2,10,2) }

X_train = df_train.drop('target', axis=1)
y_train = target_train
grid_forest = GridSearchCV(RandomForestClassifier(), parametres_forest, cv=5, n_jobs=-1)
grid_forest.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(20, 100, 10)})

In [249]:
grid_forest.best_params_

{'max_depth': 1,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 30}

In [250]:
f1_score(y_test, grid_forest.predict(X_test))

0.8333333333333333